In [2]:
from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI
import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor, create_self_ask_with_search_agent
import datetime
from langchain.agents import tool
import warnings
warnings.filterwarnings('ignore')


In [3]:
load_dotenv()
os.environ["AZURE_OPENAI_API_KEY"] = "ce53c5fce80c4503927244333e40634c"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://wdc-chat-llm.openai.azure.com/"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-4o"
os.environ["OPENAI_API_VERSION"] = "2024-06-01"

In [4]:
llm = AzureChatOpenAI(
    azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    api_version = os.environ["OPENAI_API_VERSION"],
    temperature= 0.3
)

In [5]:
@tool
def get_system_time(format: str = "%Y-%m-%d %H:%M:%S"):
    """ Returns the current date and time in the specified format """

    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime(format)
    return formatted_time

tools = [get_system_time]
query = "What is the current time in London? (You are in India). Just show the current time and not the date"

prompt_template = PromptTemplate(
    input_variables=["agent_scratchpad", "input", "tool_names", "tools"],
    template="""Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""
)

agent = create_react_agent(llm, tools, prompt_template)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
I need to determine the current time in India and then convert it to the current time in London. India is in the IST time zone (UTC+5:30), and London is in the BST time zone (UTC+1) during daylight saving time or GMT (UTC+0) otherwise.

Action: get_system_time
Action Input: '%H:%M:%S''16:40:51'I now have the current time in India, which is 16:40:51. To convert this to London time, I need to account for the time difference. India is 4 hours and 30 minutes ahead of London during BST (British Summer Time, which is UTC+1), and 5 hours and 30 minutes ahead during GMT (Greenwich Mean Time, which is UTC+0).

Since it is October, London is likely observing BST. Therefore, I need to subtract 4 hours and 30 minutes from the current time in India.

16:40:51 - 4:30:00 = 12:10:51

Final Answer: The current time in London is 12:10:51.

> Finished chain.


{'input': 'What is the current time in London? (You are in India). Just show the current time and not the date',
 'output': 'The current time in London is 12:10:51.'}